#**Importing libraries**

In [ ]:
import numpy as np
import logging 


#**Functions**

In [ ]:
class RulHandler():
    def __init__(self):
        self.logging=logging.getLogger()
        
    def compress_cycle(self,train_x,train_y):
        #compressing the train x
        train_x[train_x==0]=np.nan
        #assigning as nan
        new_train=np_empty((train_x.shape[0],train_x.shape[2],2))
        #creating new empty array of the above dimension
        for i in range(train_x.shape[2]):
            for x in range(train_x.shape[0]):
                #due to ram limitation we can't take the whole matrix 
                new_train[x,i,0] = np.nanmean(train_x[x,:,i])
                new_train[x,i,1] = np.nanstd(train_x[x,:,i])
        new_train = new_train.reshape((train_x.shape[0], train_x.shape[2]*2))
        #reshape the new train matrix so tht we can have mean and std in 1
        
        return new_train, new_test
    
    def battery_life_to_time_series(self,x,n_cycle,battery_range_cycle):
        #converting battery lifetime to x time series
        series=np.zeros((x.shape[0],n_cycle),x.shape[1])
        a=0
        for b in battery_range_cycle:
            for i in range(a,b):
                bounded_a=max(a,i+1-n_cycle)
                 series[i,0:i+1-bounded_a] = x[bounded_a:i+1]
            a = b
        return series
    
     def delete_initial(self, x, y, battery_range, soh, warmup):
        new_range = [x - warmup*(i+1) for i, x in enumerate(battery_range)]
        battery_range = np.insert(battery_range[:-1], 0, [0]) 
        indexes = [int(x+i) for x in battery_range for i in range(warmup)]
        x = np.delete(x, indexes, axis=0)
        y = np.delete(y, indexes, axis=0)
        soh = np.delete(soh.flatten(), indexes, axis=0)
        #warmup x and y
        return x, y, new_range, soh
    

     def limit_zeros(self, x, y, battery_range, soh, limit=100):
        indexes = []
        new_range = []
        a = 0
        removed = 0
        for b in battery_range:
            zeros = np.where(y[a:b,1] == 0)[0]
            zeros = zeros + a
            indexes.extend(zeros[limit:].tolist())
            removed = removed + len(zeros[limit:])
            new_range.append(b - removed)
            a = b
        x = np.delete(x, indexes, axis=0)
        y = np.delete(y, indexes, axis=0)
        soh = np.delete(soh.flatten(), indexes, axis=0)
        #limiting x and y 
        return x, y, new_range, soh
    
    
     def unify_datasets(self, x, y, battery_range, soh, m_x, m_y, m_battery_range, m_soh):
            #concanating two datasets in it 
        m_battery_range = m_battery_range + battery_range[-1]
        x = np.concatenate((x, m_x))
        y = np.concatenate((y, m_y))
        battery_range = np.concatenate((battery_range, m_battery_range))
        soh = np.concatenate((soh.flatten(), m_soh))

        #unified x,y,battery_range

        return (x, y, battery_range, soh)
    
    
     def prepare_y_future(self, battery_names, battery_n_cycle, y_soh, current, time, capacity_threshold=None, allow_negative_future = False, capacity=None):
        cycle_lenght = current.shape[1]
        battery_range_step = [x * cycle_lenght for x in battery_n_cycle]

        if capacity is None:
            battery_nominal_capacity = [ float(name.split("-")[2]) for name in battery_names]
        else:
            battery_nominal_capacity = [ capacity for name in battery_names ]
        #flatting the variables 
        current = current.ravel()
        time = time.ravel()
        capacity_integral_train = []
        a = 0
        for battery_index, b in enumerate(battery_range_step):
            integral_sum = 0
            pre_i = a
            for i in range(a,b,cycle_lenght):
                integral = np.trapz(
                    y=current[pre_i:i][current[pre_i:i]>0],
                    x=time[pre_i:i][current[pre_i:i]>0])
                integral_sum += integral
                pre_i = i
                capacity_integral_train.append(integral_sum/battery_nominal_capacity[battery_index])
            a = b
        capacity_integral_train = np.array(capacity_integral_train)
        

        y_future = []
        a = 0
        for battery_index, b in enumerate(battery_n_cycle):
           #processing range
            if capacity_threshold is None:
                index = b-1
            else:
                index = np.argmax(y_soh[a:b]<capacity_threshold[battery_nominal_capacity[battery_index]]) + a
                if index == a:
                    index = b-1
            #threshold indexing
            for i in range(a, b):
                if not allow_negative_future:
                    y = capacity_integral_train[index] - capacity_integral_train[i] if i < index else 0
                else:
                    y = capacity_integral_train[index] - capacity_integral_train[i]
                y_future.append(y)
            a = b
        y_future = np.array(y_future)
        #predicited y

        y_with_future = np.column_stack((capacity_integral_train, y_future))
        
        return y_with_future
        


#**Normalization functions**

In [ ]:
class Normalization():
        def fit(self, train):
            if len(train.shape) == 1:
                self.case = 1
                self.min = min(train)
                #minimum of the train set 
                self.max = max(train)
                #maximmum of the tain set
            elif len(train.shape) == 2:
                self.case = 2
                self.min = [min(train[:,i]) for i in range(train.shape[1])]
                self.max = [max(train[:,i]) for i in range(train.shape[1])]
            elif len(train.shape) == 3:
                self.case = 3
                self.min = [train[:,:,i].min() for i in range(train.shape[2])]
                self.max = [train[:,:,i].max() for i in range(train.shape[2])]

        def normalize(self, data):
            #normalize using x-min/(max-min) to limit data 0 to 1
            if self.case == 1:
                data = (data - self.min) / (self.max - self.min)
            elif self.case == 2:
                for i in range(data.shape[1]):
                    data[:,i] = (data[:,i] - self.min[i]) / (self.max[i] - self.min[i])
            elif self.case == 3:
                for i in range(data.shape[2]):
                    data[:,:,i] = (data[:,:,i] - self.min[i]) / (self.max[i] - self.min[i])
            return data

        def fit_and_normalize(self, train, test, val=None):
            self.fit(train)
            #fiitin the data 
            if val is not None:
                # if validation set is present 
                return self.normalize(train), self.normalize(test), self.normalize(val)
            else:
                #if validation set is not present
                return self.normalize(train), self.normalize(test)


        def denormalize(self, a):
            #vice versa of normalization
            if self.case == 1:
                a = a * (self.max - self.min) + self.min
            elif self.case == 2:
                for i in range(a.shape[1]):
                    a[:,i] = a[:,i] * (self.max[i] - self.min[i]) + self.min[i]
            elif self.case == 3:
                for i in range(a.shape[2]):
                    a[:,:,i] = a[:,:,i] * (self.max[i] - self.min[i]) + self.min[i]
            return a